In [54]:
# Use OllamaEmbeddings to create embeddings for the documents
from langchain_ollama import OllamaEmbeddings
# we are using nomic-embed-text model for the embedding as we can not directly use the models like llama2, gemma, etc.
# you can use any other model that is supported by the Ollama API
# by default, the model is set to "llama2" and the model_path is set to "llama2"
# you can change the model and model_path to any other supported models
embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [55]:
embeddings

OllamaEmbeddings(model='nomic-embed-text', base_url=None, client_kwargs={}, mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, keep_alive=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None)

In [33]:
documents = [
    "Ravi is a software engineer",
    "He is aspiring to become an AI engineer",
    "He is working on a project to build a chatbot",
    ]

embedded_docs = embeddings.embed_documents(documents)
embedded_docs

[[-0.2266027182340622,
  1.072567343711853,
  -3.434481382369995,
  -0.2065548598766327,
  0.5964371562004089,
  -0.6531354784965515,
  0.4623410105705261,
  -0.26149362325668335,
  -0.8247036933898926,
  -0.6027882695198059,
  0.07537567615509033,
  0.7165217399597168,
  1.2273935079574585,
  1.6022908687591553,
  0.10367555171251297,
  1.059245228767395,
  1.7393163442611694,
  -0.9670171141624451,
  -0.43412014842033386,
  -0.5342143774032593,
  -1.074798822402954,
  -1.0842407941818237,
  -0.29934176802635193,
  -0.2376556098461151,
  3.574960708618164,
  0.5036255121231079,
  0.07933301478624344,
  -0.2082422971725464,
  -0.9038583040237427,
  -0.7563826441764832,
  1.2084932327270508,
  -0.23158711194992065,
  -0.07658760994672775,
  -0.4483383595943451,
  -2.1885175704956055,
  -0.9623205065727234,
  1.128770112991333,
  0.8693732023239136,
  0.21974697709083557,
  0.2805514931678772,
  1.2423701286315918,
  -0.4525448679924011,
  -0.5722360014915466,
  -1.4432435035705566,
  1.

In [34]:
len(embedded_docs[0])

768

In [29]:
embedded_docs[0]
# The embedded document is a list of floats representing the embedding of the document

[-0.2266027182340622,
 1.072567343711853,
 -3.434481382369995,
 -0.2065548598766327,
 0.5964371562004089,
 -0.6531354784965515,
 0.4623410105705261,
 -0.26149362325668335,
 -0.8247036933898926,
 -0.6027882695198059,
 0.07537567615509033,
 0.7165217399597168,
 1.2273935079574585,
 1.6022908687591553,
 0.10367555171251297,
 1.059245228767395,
 1.7393163442611694,
 -0.9670171141624451,
 -0.43412014842033386,
 -0.5342143774032593,
 -1.074798822402954,
 -1.0842407941818237,
 -0.29934176802635193,
 -0.2376556098461151,
 3.574960708618164,
 0.5036255121231079,
 0.07933301478624344,
 -0.2082422971725464,
 -0.9038583040237427,
 -0.7563826441764832,
 1.2084932327270508,
 -0.23158711194992065,
 -0.07658760994672775,
 -0.4483383595943451,
 -2.1885175704956055,
 -0.9623205065727234,
 1.128770112991333,
 0.8693732023239136,
 0.21974697709083557,
 0.2805514931678772,
 1.2423701286315918,
 -0.4525448679924011,
 -0.5722360014915466,
 -1.4432435035705566,
 1.1794586181640625,
 0.16069872677326202,
 -0.0

#### Load / Chunk and Store in Vector DB

In [32]:
with open("speech.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [42]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("speech.txt")
documents = loader.load()
documents

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness be

In [43]:
# Step 2: Split into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_documents = text_splitter.split_documents(documents)
split_documents

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_co

In [44]:
# Get the embedding for the text document and store it in VectorStore DB
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents=split_documents, embedding=embeddings, persist_directory="chroma_db")
db

In [51]:
# Query the vector store for the most similar text document to the query text
query_text = "What is the speech?"

# similarity_search is the function to get the most similar text document to the query text, it returns a list of Document objects, 
# each object has a page_content and metadata attributes

# the k parameter is the number of most similar text documents to return, in this case we are getting only one document, 
# by default it returns 4 documents and we can change it to any number we want, recommended is 4 or 5 documents
# the similarity_search function uses the cosine similarity to find the most similar text document to the query text
query_result = db.similarity_search(query_text, k=1)
query_result[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy,'

In [56]:
# Query the vector store for the most similar text document to the query text
query_text = "What is the speech about?"

query_result = db.similarity_search(query_text, k=1)
query_result[0].page_content

'and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'